#### Connect to Google drive

In [18]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


#### Check for the "stock" data

In [24]:
os.listdir('/content/gdrive/Shared drives/marketdata/stock')

['2020-09-21.zip']

In [25]:
source_path = '/content/gdrive/Shared drives/marketdata/stock/'
dest_path = '/content/gdrive/Shared drives/marketdata/stock/'

#### Unzip the .zip file

In [27]:
os.chdir(source_path)

In [28]:
from zipfile import ZipFile

for zipObj in os.listdir():
    if '.zip' in zipObj:
        with ZipFile(zipObj, 'r') as zipObj:
        # Extract all the contents of zip file in current directory
            zipObj.extractall(dest_path)

#### Filter out each stock symbol

In [30]:
import pandas as pd
import numpy
import datetime as dt

In [125]:
def numtotime(num):
    ms = num % 1000
    num //= 1000
    s = num % 100
    num //= 100
    mi = num % 100
    num //= 100
    h = num % 100
    num = num // 100
    # print(h, mi, s, ms)
    return dt.datetime(2020, 9, 21, h, mi, s, ms) # TODO

In [163]:
def filter(date = "2020-09-21"):
    df_lob_dict = {}
    df_tick_dict = {}   
    df_lob = pd.read_csv(os.path.join(dest_path, date, "tse_lob.csv"), header = None)
    df_tick = pd.read_csv(os.path.join(dest_path, date, "tse_tick.csv"), header = None)
    col_lob = ['time', 'index_ori', 'stock', 'bid1_price', 'bid1_qty','bid2_price', 'bid2_qty',\
              'bid3_price', 'bid3_qty','bid4_price', 'bid4_qty','bid5_price', 'bid5_qty',\
              'ask1_qty', 'ask1_price','ask2_qty', 'ask2_price','ask3_qty', 'ask3_price',\
              'ask4_qty', 'ask4_price','ask5_qty', 'ask5_price', 'n1', 'n2']
    col_tick = ['time', 'index_ori', 'stock', 'time2', 'price', 'qty', 'n1', 'n2', 'n3', 'n4']      
    df_lob.columns = col_lob
    df_tick.columns = col_tick
    df_lob = df_lob.drop(['n1', 'n2', 'index_ori'], axis = 1)
    df_tick = df_tick.drop(['n1','n2','n3','n4','time2', 'index_ori'], axis = 1)
    df_lob = df_lob[df_lob['time'] >= 90000000]
    df_tick = df_tick[df_tick['time'] >= 90000000] 
    df_lob.stock = df_lob.stock.str.strip()
    df_tick.stock = df_tick.stock.str.strip()
    # TODO
    # stock_list = df_lob.stock.str.strip().unique()
    stock_list = ["2330", "3406"]
    # df_lob['time'] = df_lob['time'].apply(func = numtotime)
    # df_tick['time'] = df_tick['time'].apply(func = numtotime)
    # df_lob.drop
    for stock in stock_list:
        if (os.path.isdir(os.path.join(dest_path, date, stock))):
            pass
        else:
            os.mkdir(os.path.join(dest_path, date, stock))
        df_lob_dict[stock] = df_lob[df_lob.stock==stock]
        df_tick_dict[stock] = df_tick[df_tick.stock==stock]
        df_lob_dict[stock].to_csv(os.path.join(dest_path, date, stock, "tse_lob_" + stock +  ".csv"), index=False)
        df_tick_dict[stock].to_csv(os.path.join(dest_path, date, stock, "tse_tick_" + stock + ".csv"), index=False)
    return df_lob_dict, df_tick_dict

In [164]:
df_lob_dict, df_tick_dict = filter()

In [165]:
df_lob_dict

{'2330':               time stock  bid1_price  ...  ask4_price  ask5_qty  ask5_price
 85088     90000651  2330       443.0  ...       445.0       647       445.5
 85092     90000728  2330       443.0  ...       445.0       647       445.5
 85093     90000731  2330       443.0  ...       445.0       650       445.5
 85094     90000732  2330       443.0  ...       445.0       650       445.5
 85095     90000733  2330       443.0  ...       445.0       650       445.5
 ...            ...   ...         ...  ...         ...       ...         ...
 4806821  132942325  2330       440.0  ...       442.0       199       442.5
 4807239  132947338  2330       440.0  ...       442.0       199       442.5
 4807680  132952350  2330       440.0  ...       442.0       198       442.5
 4808140  132957364  2330       440.0  ...       442.0       203       442.5
 4808471  133000000  2330       440.0  ...       442.0       203       442.5
 
 [48302 rows x 22 columns],
 '3406':               time stock  bid

In [166]:
df_tick_dict

{'2330':              time stock  price   qty
 73161    90000651  2330  443.5  1370
 73163    90000745  2330  443.5     1
 73166    90000787  2330  443.0     1
 73170    90000842  2330  443.5     1
 73308    90002513  2330  443.5     1
 ...           ...   ...    ...   ...
 800123  132942325  2330  440.0  3028
 800532  132947338  2330  440.0  2958
 800964  132952350  2330  440.0  2963
 801417  132957364  2330  440.0  3027
 801742  133000000  2330  440.0  3029
 
 [10348 rows x 4 columns], '3406':              time stock  price  qty
 73650    90005977  3406  616.0  145
 73778    90007248  3406  616.0    1
 73815    90007615  3406  616.0    1
 73918    90008372  3406  616.0    1
 74103    90009488  3406  616.0    1
 ...           ...   ...    ...  ...
 800150  132942589  3406  626.0  819
 800560  132947602  3406  625.0  821
 800992  132952616  3406  624.0  728
 801454  132957631  3406  615.0  561
 801943  133000000  3406  615.0  561
 
 [4215 rows x 4 columns]}

#### For testing the code (symbol exists or not)

In [93]:
df_lob__ = pd.read_csv(os.path.join(dest_path, "2020-09-21", "tse_lob.csv"), header = None)

In [94]:
col_lob = ['time', 'index_ori', 'stock', 'bid1_price', 'bid1_qty','bid2_price', 'bid2_qty',\
              'bid3_price', 'bid3_qty','bid4_price', 'bid4_qty','bid5_price', 'bid5_qty',\
              'ask1_qty', 'ask1_price','ask2_qty', 'ask2_price','ask3_qty', 'ask3_price',\
              'ask4_qty', 'ask4_price','ask5_qty', 'ask5_price', 'n1', 'n2']

In [95]:
df_lob__.columns = col_lob

In [121]:
df_lob__.stock = df_lob__.stock.str.strip()

In [122]:
df_lob_dict = {}

In [123]:
df_lob_dict["3105"] = df_lob__[df_lob__.stock=="3105"]

In [124]:
df_lob_dict["3105"]

,time,index_ori,stock,bid1_price,bid1_qty,bid2_price,bid2_qty,bid3_price,bid3_qty,bid4_price,bid4_qty,bid5_price,bid5_qty,ask1_qty,ask1_price,ask2_qty,ask2_price,ask3_qty,ask3_price,ask4_qty,ask4_price,ask5_qty,ask5_price,n1,n2
